In [1]:
from vectorai import ViClient
import json
import xmltodict as xtd
import numpy as np
import os
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer
from joblib import Parallel, delayed
from gensim import corpora
from gensim import models
import pandas as pd
from gensim import similarities
from operator import itemgetter
import glob
import dask.bag as db
from dask.distributed import Client, progress
from vectorai.models.deployed import ViText2Vec

In [15]:
client = Client(n_workers=6, threads_per_worker=2)

client

B:\Anaconda3\lib\site-packages\distributed\dashboard\core.py:79: UserWarning: 
Failed to start diagnostics server on port 8787. [WinError 10048] Solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:52184 Dashboard: http://127.0.0.1:52187/status,Cluster Workers: 6 Cores: 12 Memory: 17.13 GB


In [16]:
username = 'cesar-cabeza'
api_key = 'amdXU0VYVUJXN1RRYmVKWGUwcG46Ni1zRW15R01SNFNmLUZ4Ry00MHFmZw'

vi_client = ViClient(username=username, api_key = api_key)
text_encoder = ViText2Vec(username, api_key)

Logged in. Welcome cesar-cabeza. To view list of available collections, call list_collections() method.


In [17]:
vi_client.delete_collection("covid-19")

{'status': 'complete', 'message': 'covid-19 deleted'}

In [18]:
path_test = "B:/document_parser/document_parses/test"

In [24]:
def load_corpus(files_path, client, text_encoder):
    files = glob.glob(files_path + "/*.json")
    stopset = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    b = db.from_sequence(files).map(preprocess_json, stopset, stemmer)
    client.insert_documents("covid-19", b.compute(), models={
                            "abstract": text_encoder.encode, "title": text_encoder.encode}, workers = 12, chunksize = 20)

In [25]:
def preprocess_json(file_path, stopset, stemmer):
    with open(file_path) as file:
        file_json = json.load(file)
    returned = preprocess_document(file_json, stopset, stemmer)
    return returned

In [26]:
def preprocess_document(file, stopset, stemmer):
    returned = {}
    returned["_id"] = file["paper_id"]
    title = file["metadata"]["title"]
    abstract = " ".join([paragraph["text"] for paragraph in file["abstract"]])
    body = " ".join([paragraph["text"] for paragraph in file["body_text"]])
    returned["title"] = title
    returned["abstract"] = abstract#preprocess_document_part(abstract, stopset, stemmer)
    returned["body"] = body#preprocess_document_part(body, stopset, stemmer)
    return returned

In [27]:
def preprocess_document_part(part, stopset, stemmer):
    if part != "":  # actually has content
        tokens = [word.lower() for word in wordpunct_tokenize(
            part) if word.lower() not in stopset and len(word) > 2]
        stems = [stemmer.stem(token) for token in tokens]
        return " ".join(stems)
    return "no content"

In [ ]:
load_corpus(path_test, vi_client, text_encoder)

In [ ]:
vi_client.collection_schema("covid-19")

In [ ]:
stopset = set(stopwords.words("english"))
stemmer = PorterStemmer()
tokens = [word.lower() for word in wordpunct_tokenize("what is the origin of coronavirus") if word.lower() not in stopset and len(word) > 2]
stems = [stemmer.stem(token) for token in tokens]
query = " ".join(stems)

In [ ]:
advanced_search_query = {
    'text' : {'vector': text_encoder.encode("what is the origin of coronavirus"), 'fields' : ['abstract_vector_', "title_vector_"]}
}
results = vi_client.advanced_search('covid-19', advanced_search_query, page_size=10)

In [ ]:
results

In [ ]:
[(result["title"],result["_search_score"]) for result in results["results"]]

In [ ]:
vi_client.random_documents("covid-19")

In [11]:
files = glob.glob(path_test + "/*.json")
stopset = set(stopwords.words("english"))
stemmer = PorterStemmer()
b = db.from_sequence(files).map(preprocess_json, stopset, stemmer)

In [14]:
b.take(100)

({'_id': '000a0fc8bbef80410199e690191dc3076a290117',
  'title': 'PfSWIB, a potential chromatin regulator for var gene regulation and parasite development in Plasmodium falciparum',
  'abstract': 'Background: Various transcription factors are involved in the process of mutually exclusive expression and clonal variation of the Plasmodium multigene (var) family. Recent studies revealed that a P. falciparum SWI/SNF-related matrix-associated actin-dependent regulator of chromatin (PfSWIB) might trigger stage-specific programmed cell death (PCD), and was not only crucial for the survival and development of parasite, but also had profound effects on the parasite by interacting with other unknown proteins. However, it remains unclear whether PfSIWB is involved in transcriptional regulation of this virulence gene and its functional properties. Methods: A conditional knockdown system "PfSWIB-FKBP-LID" was introduced to the parasite clone 3D7, and an integrated parasite line "PfSWIB-HA-FKBP-LID" 